In [49]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [50]:
# To detect diseases from symptoms and recommend Ayurvedic medicine

In [51]:
import pandas as pd # data preprocessing
import itertools # confusion matrix
import string
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
# To show all the rows of pandas dataframe
pd.set_option('display.max_rows', None)

In [52]:
df = pd.read_csv("drugsComTrain_raw.tsv",sep="\t")

In [53]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BaAr349\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
df.head(10)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0,"March 14, 2015",32
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0,"August 9, 2016",11
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1


In [55]:
# df.review[62]

In [56]:
df.drugName.value_counts().sort_values(ascending=False).head(10)

drugName
Levonorgestrel                        3657
Etonogestrel                          3336
Ethinyl estradiol / norethindrone     2850
Nexplanon                             2156
Ethinyl estradiol / norgestimate      2117
Ethinyl estradiol / levonorgestrel    1888
Phentermine                           1543
Sertraline                            1360
Escitalopram                          1292
Mirena                                1242
Name: count, dtype: int64

In [57]:
df[df['condition']=='Anxiety'].sort_values(by=['rating','usefulCount'],ascending=False).head(10)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
62592,201777,Alprazolam,Anxiety,"""This drug has changed my life. I have always ...",10.0,"May 26, 2008",526
78849,8088,Xanax,Anxiety,"""This drug has changed my life. I have always ...",10.0,"May 26, 2008",526
11501,160851,Buspirone,Anxiety,"""This medication is unbelievable. After 3 days...",10.0,"April 27, 2009",520
145742,204615,BuSpar,Anxiety,"""This medication is unbelievable. After 3 days...",10.0,"April 27, 2009",520
30257,50489,Gabapentin,Anxiety,"""Neurontin has changed my life dramatically. I...",10.0,"May 24, 2012",500
79489,96532,Neurontin,Anxiety,"""Neurontin has changed my life dramatically. I...",10.0,"May 24, 2012",500
81546,201628,Alprazolam,Anxiety,"""Xanax is a miracle for me. I can&#039;t live ...",10.0,"December 9, 2008",401
92996,7928,Xanax,Anxiety,"""Xanax is a miracle for me. I can&#039;t live ...",10.0,"December 9, 2008",401
52126,160844,Buspirone,Anxiety,"""Buspar has been very effective for me in trea...",10.0,"April 29, 2008",388
76292,204657,BuSpar,Anxiety,"""Buspar has been very effective for me in trea...",10.0,"April 29, 2008",388


In [58]:
df.columns

Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')

In [59]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df = df.dropna(subset=['condition'])
df.isnull().sum()

drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64

In [60]:
df.condition.value_counts(ascending=False).head(10)

condition
Birth Control      28788
Depression          9069
Pain                6145
Anxiety             5904
Acne                5588
Bipolar Disorde     4224
Insomnia            3673
Weight Loss         3609
Obesity             3568
ADHD                3383
Name: count, dtype: int64

In [61]:
df_train = df[(df['condition'] == 'Birth Control') |
              (df['condition'] == 'Depression') |
              (df['condition'] == 'Pain') |
              (df['condition'] == 'Anxiety') |
              (df['condition'] == 'Acne')]


In [62]:
df_train.isnull().sum()
# dt = df_train[df_train['condition']=='Pain']

drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64

In [63]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BaAr349\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
k=0
birth = []
for i in df[df['condition']=='Anxiety']['review']:
    birth.append(i)
    k+=1
    if k==5:
        break
birth

['"Was on this med for 5 years. Worked fine but not great. Stopped the panic attacks and gave me relief of every day nervousness that I was experiencing. Took months for the side effects to diminish when I first started taking it. What did not leave - insomnia, night sweats, lack of sex drive and poor orgasms (or no orgasms at all)."',
 '"I used it at bed time along with Paxil. I have anxiety and at times insomnia. The lower the dose(@ 7.5 mg), the more it becomes a sleep aid. It worked well for both anxiety and insomnia."',
 '"Have tried Paxil and Lexapro which sent me into hyper-panic and GAD. I was about to give up on medicines and accept my miserable anxiety as a fact of life when my Doctor recommended Buspirone. I noticed my anxiety began to taper the second day I took the medication. No real side effects to speak of through my first week. The one thing it has delivered for me, HOPE!"',
 '"I have been suffering with this anxiety problem for years and it was starting to interfere w

In [65]:
X = df_train[['condition', 'review']]

In [66]:
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def clean(text):
    html = BeautifulSoup(text,'html.parser').get_text()

    words = re.sub(r'[^a-zA-Z]',' ',html)

    lowercase = words.lower().split()

    meaningfulwords = [w for w in lowercase if not w in stop]

    lemma = [lemmatizer.lemmatize(w) for w in meaningfulwords]

    return (' '.join(lemma))

X['cleanReview'] = df_train['review'].apply(clean)

# import re
# from tqdm import tqdm
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer

# def decontracted(phrase):
#     # specific
#     phrase = re.sub(r"won't", "will not", phrase)
#     phrase = re.sub(r"can\'t", "can not", phrase)
#     # general
#     phrase = re.sub(r"n\'t", " not", phrase)
#     phrase = re.sub(r"\'re", " are", phrase)
#     phrase = re.sub(r"\'s", " is", phrase)
#     phrase = re.sub(r"\'d", " would", phrase)
#     phrase = re.sub(r"\'ll", " will", phrase)
#     phrase = re.sub(r"\'t", " not", phrase)
#     phrase = re.sub(r"\'ve", " have", phrase)
#     phrase = re.sub(r"\'m", " am", phrase)
#     return phrase

# def preprocess_text(text_data):

#     text_data = decontracted(text_data)
    
#     text_data = text_data.replace('\n',' ')
#     text_data = text_data.replace('\r',' ')
#     text_data = text_data.replace('\t',' ')
#     text_data = text_data.replace('-',' ')
#     text_data = text_data.replace("/",' ')
#     text_data = text_data.replace(">",' ')
#     text_data = text_data.replace('"',' ')
#     text_data = text_data.replace('?',' ')
#     return text_data
  
# # loading stop words from nltk library

# stop_words = set(stopwords.words('english'))
# stemmer = SnowballStemmer('english')

# #removing 'no' from the stop words list as there is an importance of 'side effects' and 'no side effects' in review
# stop_words.remove('no')

# def nlp_preprocessing(review):
   
#     if type(review) is not int:
#         string = ""
#         review = preprocess_text(review)
#         review = re.sub('[^a-zA-Z]', ' ', review)
#         review = re.sub('\s+',' ', review)
#         review = review.lower()
#         for word in review.split():
#             if not word in stop_words:
#                 word = stemmer.stem(word)
#                 string += word + " "
#         return string 
      
# X['cleaned_review'] = X['review'].apply(nlp_preprocessing)


C:\Users\BaAr349\AppData\Local\Temp\ipykernel_36304\1989213923.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  html = BeautifulSoup(text,'html.parser').get_text()
C:\Users\BaAr349\AppData\Local\Temp\ipykernel_36304\1989213923.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cleanReview'] = df_train['review'].apply(clean)


In [67]:
X = X.reset_index(drop=True)
print(X.cleanReview[10])

pain management doctor put butrans patch week ago mg dose first box four lifesaver agony work able sleep two weekend two year hoping bump ten mg dose soon cut norco chronic pain many year many medicine including oxycontin patch best far


In [68]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
X['sentiment_score'] = [sid.polarity_scores(v)['compound'] for v in X['review']]
X['sentiment_score_clean'] = [sid.polarity_scores(v)['compound'] for v in X['cleanReview']]

In [69]:
X.head()

,condition,review,cleanReview,sentiment_score,sentiment_score_clean
0,Birth Control,"""I used to take another oral contraceptive, wh...",used take another oral contraceptive pill cycl...,0.7962,0.8176
1,Birth Control,"""This is my first time using any form of birth...",first time using form birth control glad went ...,0.7184,0.5267
2,Birth Control,"""I had been on the pill for many years. When m...",pill many year doctor changed rx chateal effec...,-0.9010,0.8179
3,Depression,"""I have taken anti-depressants for years, with...",taken anti depressant year improvement mostly ...,0.0147,-0.0000
4,Birth Control,"""Started Nexplanon 2 months ago because I have...",started nexplanon month ago minimal amount con...,-0.4309,0.4458


In [70]:
x = X['cleanReview']
y = X['condition']

from sklearn.model_selection import train_test_split

In [71]:
X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y,test_size=0.25, random_state=42)

In [72]:

import itertools # confusion matrix
import string
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline

# To show all the rows of pandas dataframe
pd.set_option('display.max_rows', None)



In [73]:

# Modify the TF-IDF vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, ngram_range=(1, 2))

# Fit and transform the training data
train_vectoriser = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
test_vectoriser = tfidf_vectorizer.transform(X_test)

# Print the shape of the transformed test data
print("Test vectorizer shape:", test_vectoriser.shape)

print("Train vectorizer shape:", train_vectoriser.shape)


Test vectorizer shape: (13874, 493710)
Train vectorizer shape: (41620, 493710)


In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Initialize the RandomForestClassifier with specified parameters
randomforest = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, n_jobs=-1)

# Fit the model on the training data
randomforest.fit(train_vectoriser , y_train)

# Make predictions on the test data
prediction = randomforest.predict(test_vectoriser)

# Calculate the accuracy of the model
score = metrics.accuracy_score(prediction,y_test)
print(score)

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cf= confusion_matrix(prediction , y_test)
print(cf)

0.5187400893758108
[[   0    0    0    0    0]
 [   0    0    0    0    0]
 [1397 1476 7197 2268 1536]
 [   0    0    0    0    0]
 [   0    0    0    0    0]]


In [75]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the DecisionTreeClassifier with specified parameters
decision_tree = DecisionTreeClassifier(max_depth=10, min_samples_split=5)

# Fit the model on the training data
decision_tree.fit(train_vectoriser , y_train)

# Make predictions on the test data
prediction_dt = decision_tree.predict(test_vectoriser)

# Calculate the accuracy of the model
score_dt = metrics.accuracy_score(prediction_dt, y_test)
print("Decision Tree Accuracy:", score_dt)

# Calculate the confusion matrix
cf_dt = confusion_matrix(prediction_dt, y_test)
print("Confusion Matrix for Decision Tree:")
print(cf_dt)


Decision Tree Accuracy: 0.8133919561770218
Confusion Matrix for Decision Tree:
[[1069    5  137   10    5]
 [   1  897   50  219   20]
 [ 304  305 6833  635  336]
 [  15  258  105 1365   54]
 [   8   11   72   39 1121]]


In [76]:
import lightgbm as lgb

# Initialize the LightGBM Classifier with specified parameters
lgbm = lgb.LGBMClassifier(num_leaves=50, max_depth=10, learning_rate=0.1, n_estimators=100)

# Fit the model on the training data
lgbm.fit(train_vectoriser , y_train)

# Make predictions on the test data
prediction_lgbm = lgbm.predict(test_vectoriser)

# Calculate the accuracy of the model
score_lgbm = metrics.accuracy_score(prediction_lgbm, y_test)
print("LightGBM Accuracy:", score_lgbm)

# Calculate the confusion matrix
cf_lgbm = confusion_matrix(prediction_lgbm, y_test)
print("Confusion Matrix for LightGBM:")
print(cf_lgbm)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.324990 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 458743
[LightGBM] [Info] Number of data points in the train set: 41620, number of used features: 14586
[LightGBM] [Info] Start training from score -2.295641
[LightGBM] [Info] Start training from score -2.240633
[LightGBM] [Info] Start training from score -0.656304
[LightGBM] [Info] Start training from score -1.811511
[LightGBM] [Info] Start training from score -2.200570
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [77]:
from catboost import CatBoostClassifier

# Initialize the CatBoostClassifier with specified parameters
catboost = CatBoostClassifier(iterations=20, depth=6, learning_rate=0.1)

# Fit the model on the training data
catboost.fit(train_vectoriser , y_train)

# Make predictions on the test data
prediction_catboost = catboost.predict(test_vectoriser)

# Calculate the accuracy of the model
score_catboost = metrics.accuracy_score(prediction_catboost, y_test)
print("CatBoost Accuracy:", score_catboost)

# Calculate the confusion matrix
cf_catboost = confusion_matrix(prediction_catboost, y_test)
print("Confusion Matrix for CatBoost:")
print(cf_catboost)


0:	learn: 1.4047537	total: 5.78s	remaining: 1m 49s
1:	learn: 1.2753021	total: 10.6s	remaining: 1m 35s
2:	learn: 1.1749614	total: 15.5s	remaining: 1m 27s
3:	learn: 1.0907268	total: 20.4s	remaining: 1m 21s
4:	learn: 1.0217491	total: 25.3s	remaining: 1m 15s
5:	learn: 0.9636106	total: 30.2s	remaining: 1m 10s
6:	learn: 0.9141734	total: 35.2s	remaining: 1m 5s
7:	learn: 0.8711381	total: 40.1s	remaining: 1m
8:	learn: 0.8353694	total: 45s	remaining: 55s
9:	learn: 0.8030681	total: 50.1s	remaining: 50.1s
10:	learn: 0.7735006	total: 55s	remaining: 45s
11:	learn: 0.7495724	total: 60s	remaining: 40s
12:	learn: 0.7263325	total: 1m 4s	remaining: 35s
13:	learn: 0.7057605	total: 1m 9s	remaining: 30s
14:	learn: 0.6889380	total: 1m 14s	remaining: 24.9s
15:	learn: 0.6728046	total: 1m 19s	remaining: 19.9s
16:	learn: 0.6567792	total: 1m 24s	remaining: 14.9s
17:	learn: 0.6437666	total: 1m 29s	remaining: 9.94s
18:	learn: 0.6289889	total: 1m 34s	remaining: 4.96s
19:	learn: 0.6165040	total: 1m 39s	remaining: 0us

In [78]:
multinb = MultinomialNB()
multinb.fit(train_vectoriser , y_train)
prediction = multinb.predict(test_vectoriser)
score = metrics.accuracy_score(prediction,y_test)
print(score)
from sklearn.metrics import confusion_matrix
cf= confusion_matrix(prediction , y_test)
print(cf)

0.6660660227764164
[[ 282    0    0    0    0]
 [   0  242    0    4    0]
 [1113  966 7197 1311  954]
 [   2  268    0  951   13]
 [   0    0    0    2  569]]


In [79]:
passive_tf = PassiveAggressiveClassifier()
passive_tf.fit(train_vectoriser , y_train)
pred = passive_tf.predict(test_vectoriser)
score = metrics.accuracy_score(pred,y_test)
print(score)


0.9564653308346548


In [80]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Calculate performance metrics for PassiveAggressiveClassifier
pa_accuracy = metrics.accuracy_score(pred, y_test)
pa_precision = precision_score(y_test, pred, average='weighted')
pa_recall = recall_score(y_test, pred, average='weighted')
pa_f1 = f1_score(y_test, pred, average='weighted')
# pa_auc = roc_auc_score(y_test, pred)

# Calculate performance metrics for MultinomialNB
nb_accuracy = metrics.accuracy_score(prediction, y_test)
nb_precision = precision_score(y_test, prediction, average='weighted')
nb_recall = recall_score(y_test, prediction, average='weighted')
nb_f1 = f1_score(y_test, prediction, average='weighted')
# nb_auc = roc_auc_score(y_test, prediction)


# Calculate performance metrics for CatBoostClassifier
catboost_accuracy = metrics.accuracy_score(prediction_catboost, y_test)
catboost_precision = precision_score(y_test, prediction_catboost, average='weighted')
catboost_recall = recall_score(y_test, prediction_catboost, average='weighted')
catboost_f1 = f1_score(y_test, prediction_catboost, average='weighted')
# catboost_auc = roc_auc_score(y_test, prediction_catboost)

# Calculate performance metrics for LightGBMClassifier
lgbm_accuracy = metrics.accuracy_score(prediction_lgbm, y_test)
lgbm_precision = precision_score(y_test, prediction_lgbm, average='weighted')
lgbm_recall = recall_score(y_test, prediction_lgbm, average='weighted')
lgbm_f1 = f1_score(y_test, prediction_lgbm, average='weighted')
# lgbm_auc = roc_auc_score(y_test, prediction_lgbm)

# Calculate performance metrics for DecisionTreeClassifier
dt_accuracy = metrics.accuracy_score(prediction_dt, y_test)
dt_precision = precision_score(y_test, prediction_dt, average='weighted')
dt_recall = recall_score(y_test, prediction_dt, average='weighted')
dt_f1 = f1_score(y_test, prediction_dt, average='weighted')
# dt_auc = roc_auc_score(y_test, prediction_dt)

# Calculate performance metrics for RandomForestClassifier
rf_accuracy = metrics.accuracy_score(prediction, y_test)
rf_precision = precision_score(y_test, prediction, average='weighted')
rf_recall = recall_score(y_test, prediction, average='weighted')
rf_f1 = f1_score(y_test, prediction, average='weighted')
# rf_auc = roc_auc_score(y_test, prediction)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Classifier': ['PassiveAggressive', 'MultinomialNB', 'CatBoost', 'LightGBM', 'DecisionTree', 'RandomForest'],
    'Accuracy': [pa_accuracy, nb_accuracy, catboost_accuracy, lgbm_accuracy, dt_accuracy, rf_accuracy],
    'Precision': [pa_precision, nb_precision, catboost_precision, lgbm_precision, dt_precision, rf_precision],
    'Recall': [pa_recall, nb_recall, catboost_recall, lgbm_recall, dt_recall, rf_recall],
    'F1 Score': [pa_f1, nb_f1, catboost_f1, lgbm_f1, dt_f1, rf_f1],
    # 'AUC': [pa_auc, nb_auc, catboost_auc, lgbm_auc, dt_auc, rf_auc]
})

# Print the DataFrame
results_sorted = results.sort_values(by='Accuracy', ascending=False)

# Print the sorted DataFrame
print(results_sorted)


          Classifier  Accuracy  Precision    Recall  F1 Score
0  PassiveAggressive  0.956465   0.956858  0.956465  0.956360
3           LightGBM  0.935995   0.937354  0.935995  0.935939
4       DecisionTree  0.813392   0.812889  0.813392  0.806728
2           CatBoost  0.808851   0.807544  0.808851  0.801866
1      MultinomialNB  0.666066   0.765140  0.666066  0.610787
5       RandomForest  0.666066   0.765140  0.666066  0.610787


In [81]:
from sklearn.metrics import confusion_matrix
cf= confusion_matrix(pred , y_test)
print(cf)

[[1305    0   24    3    3]
 [   0 1252    6  109   17]
 [  81   21 7132   12    9]
 [   7  187   29 2128   54]
 [   4   16    6   16 1453]]


In [82]:
def most_informative_feature_for_class(vectorizer, classifier, classlabel, n=10):
    labelid = list(classifier.classes_).index(classlabel)
    feature_names = vectorizer.get_feature_names_out()
    topn = sorted(zip(classifier.coef_[labelid], feature_names))[-n:]
    for coef, feat in topn:
        print (classlabel, feat, coef)
most_informative_feature_for_class(tfidf_vectorizer, passive_tf, 'Pain')

Pain narcotic 3.5676102013049213
Pain dilaudid 3.6243578311545215
Pain opana 3.7032558968583444
Pain norco 4.043918442065478
Pain oxycontin 4.2431487632241085
Pain relief 4.351793540758756
Pain nucynta 4.552689873456806
Pain surgery 4.872233503039562
Pain percocet 5.158850876557711
Pain pain 10.461970107874446


In [83]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(passive_tf, file)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)


In [84]:
def top_drugs_extractor(condition):
    return df[(df['rating'] >= 9) & (df['usefulCount'] >= 90) & (df['condition'] == condition)] \
        .sort_values(by=['rating', 'usefulCount'], ascending=[False, False])['drugName'].head(7).tolist()
def predict_and_print(text, label):
    target_conditions = {
        "Depression": "Depression",
        "Acne": "Acne",
        "Anxiety":"Anxiety",
        # "Pain":"Pain",
        "Birth Control": "Birth Control"
    }

    target = target_conditions.get(label, "Unknown Condition")

    if target != "Unknown Condition":
        top_drugs = top_drugs_extractor(target)
        top_drugs = list(set(top_drugs))
        print("text:", text, "\nCondition:", target)
        print("Top 3 Suggested Drugs:")
        if len(top_drugs):
            print(top_drugs[0])
            print(top_drugs[1])
            print(top_drugs[2])
        else:
            print("Not enough drugs for the specified condition.")
    else:
        print("Can't recommend drugs when the condition is unknown.")
    print()

# Sample sentences for recommending drugs
sentences = [
    # "pain management doctor put  butrans patch week ago mg dose first box four lifesaver agony work able sleep two weekend two year hoping bump ten mg dose soon cut norco chronic pain many year many medicine including oxycontin patch best far",
    "Have tried Paxil and Lexapro which sent me into hyper-panic and GAD. I was about to give up on medicines and accept my miserable anxiety as a fact of life when my Doctor recommended Buspirone. I noticed my anxiety began to taper the second day I took the medication. No real side effects to speak of through my first week. The one thing it has delivered for me, HOPE!",
    "1 week on Zoloft for anxiety and mood swings. I take 50mg in the mornings with my breakfast. Nausea on day one  but that subsided as the week went on.  I get the jitters about 2 hrs after taking it followed by yawning. I feel much better though and less angry/stressed.",
    "I&#039;m 30 years old.  I started having really bad skin about 2 years ago from maybe stress in my life.  But, I tried EVERYTHING to get rid of it and it just got worse.  I finally went to a dermatologist and she put me on spironolactone.  My skin started getting better in about two weeks. I LOVED it. I&#039;ve been taking it for about 6 months and I get compliments from Makeup artist and models (I&#039;m a hairstylist) about how good my skin is.  Another plus for me is my breasts went from a small B to a C! I decided to stop taking it for a week and my breasts went small again and my skin is getting a little oily. So I&#039;m going back on. I only take one a day.",
    "I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas."
]

tfidf_trigram = tfidf_vectorizer.transform(sentences)
predictions = passive_tf.predict(tfidf_trigram)

for text, label in zip(sentences, predictions):
    predict_and_print(text, label)



text: Have tried Paxil and Lexapro which sent me into hyper-panic and GAD. I was about to give up on medicines and accept my miserable anxiety as a fact of life when my Doctor recommended Buspirone. I noticed my anxiety began to taper the second day I took the medication. No real side effects to speak of through my first week. The one thing it has delivered for me, HOPE! 
Condition: Anxiety
Top 3 Suggested Drugs:
Buspirone
Gabapentin
Neurontin

text: 1 week on Zoloft for anxiety and mood swings. I take 50mg in the mornings with my breakfast. Nausea on day one  but that subsided as the week went on.  I get the jitters about 2 hrs after taking it followed by yawning. I feel much better though and less angry/stressed. 
Condition: Depression
Top 3 Suggested Drugs:
Vilazodone
Zoloft
Pristiq

text: I&#039;m 30 years old.  I started having really bad skin about 2 years ago from maybe stress in my life.  But, I tried EVERYTHING to get rid of it and it just got worse.  I finally went to a derma